In [ ]:
import tensorflow as tf
from tensorflow import keras

#define linear regression model using tf.keras.Model
class LinearRegression(tf.keras.Model):
  def __init__(self):
    super(LinearRegression, self).__init__()
    self.linear_layer = tf.keras.layers.Dense(1, activation=None)

  def call(self, x):
    y_pred = self.linear_layer(x)

    return y_pred

#define loss function
@tf.function
def mse_loss(y_pred, y):
  return tf.reduce_mean(tf.square(y_pred - y)) #MSE loss function => mean{(y'-y)^2}

#define gradient descent optimizer
oprimizer = tf.optimizers.SGD(0.01)

#define folder address to save tensorboard summary information
summary_writer = tf.summary.create_file_writer('./tensorboard_log')

#define function for optimization
